In [1]:
import json
import pandas as pd
import xlrd
import os
import re

# Salto

In [2]:
with open('../data/data_salto/concursos/concursos_salto_nacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 124
La longitud de Categoría es: 124
La longitud de Provincia es: 124
La longitud de Localidad es: 124
La longitud de Disciplina es: 124
La longitud de Federación es: 124
La longitud de Resultados es: 124
La longitud de País es: 124
La longitud de Inicio es: 124
La longitud de Final es: 124
La longitud de Ambito es: 124


In [3]:
with open('../data/data_salto/concursos/concursos_salto_nacional_2025.json', 'r', encoding='utf-8') as file:
    datos_concursos = json.load(file)

concursos_ = pd.DataFrame(datos_concursos)
concursos_

,Nombre,Categoría,Provincia,Localidad,Disciplina,Federación,Resultados,País,Inicio,Final,Ambito
0,CSN3* Invierno,CSN***,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Ver,Epaña,10/01/2025,12/01/2025,Nacional
1,SPRING MET I 2025 YH*,CSI YH*,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,14/01/2025,16/01/2025,Internacional
2,SPRING MET I 2025 - SILVER 1*,CSI*,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,16/01/2025,18/01/2025,Internacional
3,SPRING MET I 2025 - GOLD 2*,CSI**,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,16/01/2025,19/01/2025,Internacional
4,CSIYH* - 1st Week Andalucía Pre-Sunshine Tour ...,CSI YH*,Cádiz,Dehesa Montenmedio,Salto de Obstáculos,Federación Hípica Andaluza,Ver,Epaña,16/01/2025,19/01/2025,Internacional
...,...,...,...,...,...,...,...,...,...,...,...
119,SPRING MET IV - YH*,CSI YH*,Valencia,Ceon,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,22/04/2025,24/04/2025,Internacional
120,SPRING MET IV - CSI1* SILVER,CSI*,Valencia,Ceon,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,24/04/2025,26/04/2025,Internacional
121,SPRING MET IV - CSI3* GOLD,CSI***,Valencia,Ceon,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,24/04/2025,27/04/2025,Internacional
122,SPRING MET IV - CSI1* BRONZE,CSI*,Valencia,Ceon,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,25/04/2025,27/04/2025,Internacional


In [4]:
len(concursos_["Nombre"].unique())

78

In [5]:
def creacion_columnas(df, columna_crear, columna, coso = "paises"):
    posiciones = ["RET", "ELI", "NOP"]

    df[columna] = df[columna].astype(str)
    
    if coso == "paises":
        df[columna_crear] = df[columna].apply(lambda x: x.split("(")[1].strip(")") if x.endswith(")") else "ESP")
        df[columna] = df[columna].apply(lambda x: x.split("(")[0].strip() if x.endswith(")") else x)
    elif coso == "Estado": 
        if "Puntuacion" in df.columns:  
            df["Posicion"] = df.apply(lambda x: "NOP" if pd.isna(x["Puntuacion"]) else x["Posicion"], axis=1)
        df[columna_crear] = df[columna].apply(lambda x: x if x in posiciones else "FIN")
    elif coso == "Premio":
        df[columna_crear] = df[columna].apply(lambda x: False if x == 0 else True)
        
def modificaciones_generales(dataframe):

    dataframe["Prueba"] = dataframe.iloc[3,1]
    dataframe["Fecha_prueba"] = dataframe.iloc[2,1]
    dataframe["Concurso"] = dataframe.iloc[1,1].split("(")[0].strip()
    dataframe = dataframe.iloc[9:].reset_index(drop = True)
    dataframe = dataframe.rename(columns = {"Unnamed: 0": "Posicion", 
                                                 "Unnamed: 1": "Lic_jinete",
                                                 "Unnamed: 2": "Jinete", 
                                                 "Unnamed: 3": "Lic_caballo", 
                                                 "Unnamed: 4": "Caballo",
                                                 "Unnamed: 5": "Raza_caballo",
                                                 "Unnamed: 6": "Puntuacion",
                                                 "Unnamed: 7": "Dinero_premio"})
    dataframe["Raza_caballo"] = dataframe["Raza_caballo"].apply(lambda x: None if x == "--" else x)
    creacion_columnas(dataframe, "Pais_jinete", "Jinete")
    creacion_columnas(dataframe, "Pais_caballo", "Caballo")
    creacion_columnas(dataframe, "Estado", "Posicion", coso = "Estado")
    creacion_columnas(dataframe, "Premio", "Dinero_premio", coso = "Premio")
    dataframe = dataframe.reindex(columns = ['Estado','Posicion', 'Lic_jinete', 'Jinete', 'Pais_jinete', 'Lic_caballo', 'Caballo',
              'Raza_caballo', 'Pais_caballo', 'Puntuacion', 'Premio','Dinero_premio', 'Prueba', 'Fecha_prueba',
              'Concurso'])

    return dataframe

def parse_puntuacion_tupla(puntuacion_str):
    """
    Parsea la cadena de puntuación y devuelve una tupla con los valores
    para las nuevas columnas.
    """
    resultados = str(puntuacion_str).split('\n')
    ptos_obs = [None] * 3
    ptos_tiempo = [None] * 3
    tiempo = [None] * 3

    for i, resultado in enumerate(resultados[:3]):
        obs_match = re.match(r'(\d+)\((\d+) Obs - (\d+) Tpo\)/([\d,]+)', resultado)
        simple_match = re.match(r'(\d+)/([\d,]+)', resultado)
        solo_numero_match = re.match(r'([\d,]+)', resultado)
        numero_barra_texto = re.match(r'(\d+)/([\d,]+)\n[A-Z]+', resultado)
        numero_obs_barra_texto = re.match(r'(\d+)\((\d+) Obs - (\d+) Tpo\)/([\d,]+)\n[A-Z]+', resultado)
        numero_barra_numero_barra_numero = re.match(r'(\d+)\n(\d+)/([\d,]+)', resultado)
        numero_barra_numero_barra_numero_obs = re.match(r'(\d+)\n(\d+)\((\d+) Obs - (\d+) Tpo\)/([\d,]+)', resultado)
        numero_obs_barra_numero_barra_numero = re.match(r'(\d+)\((\d+) Obs - (\d+) Tpo\)\n(\d+)/([\d,]+)', resultado)
        numero_barra_solo_numero = re.match(r'(\d+)\n([\d,]+)', resultado)

        if obs_match:
            ptos_obs[i] = int(obs_match.group(2))
            ptos_tiempo[i] = int(obs_match.group(3))
            tiempo[i] = float(obs_match.group(4).replace(',', '.'))
        elif simple_match:
            ptos_obs[i] = int(simple_match.group(1))
            tiempo[i] = float(simple_match.group(2).replace(',', '.'))
        elif solo_numero_match:
            ptos_tiempo[i] = float(solo_numero_match.group(0).replace(',', '.'))
        elif numero_barra_texto:
            ptos_obs[i] = int(numero_barra_texto.group(1))
            tiempo[i] = float(numero_barra_texto.group(2).replace(',', '.'))
        elif numero_obs_barra_texto:
            ptos_obs[i] = int(numero_obs_barra_texto.group(2))
            ptos_tiempo[i] = int(numero_obs_barra_texto.group(3))
            tiempo[i] = float(numero_obs_barra_texto.group(4).replace(',', '.'))
        elif numero_barra_numero_barra_numero:
            if i == 0:
                pass # El primer número solo no se mapea directamente según tu ejemplo "number\number/number"
            elif i == 1:
                ptos_obs[i] = int(numero_barra_numero_barra_numero.group(2))
                tiempo[i] = float(numero_barra_numero_barra_numero.group(3).replace(',', '.'))
        elif numero_barra_numero_barra_numero_obs:
            if i == 0:
                pass # El primer número solo no se mapea directamente
            elif i == 1:
                obs_match_r2 = re.match(r'(\d+)\((\d+) Obs - (\d+) Tpo\)/([\d,]+)', resultado.split('\n')[1])
                if obs_match_r2:
                    ptos_obs[i] = int(obs_match_r2.group(2))
                    ptos_tiempo[i] = int(obs_match_r2.group(3))
                    tiempo[i] = float(obs_match_r2.group(4).replace(',', '.'))
        elif numero_obs_barra_numero_barra_numero:
            if i == 0:
                ptos_obs[i] = int(numero_obs_barra_numero_barra_numero.group(2))
                ptos_tiempo[i] = int(numero_obs_barra_numero_barra_numero.group(3))
            elif i == 1:
                simple_match_r2 = re.match(r'(\d+)/([\d,]+)', resultado.split('\n')[1])
                if simple_match_r2:
                    ptos_obs[i] = int(simple_match_r2.group(1))
                    tiempo[i] = float(simple_match_r2.group(2).replace(',', '.'))
        elif numero_barra_solo_numero:
            if i == 0:
                ptos_obs[i] = int(numero_barra_solo_numero.group(1))
            elif i == 1:
                ptos_tiempo[i] = float(numero_barra_solo_numero.group(2).replace(',', '.'))


    return (ptos_obs[0], ptos_tiempo[0], tiempo[0],
            ptos_obs[1], ptos_tiempo[1], tiempo[1],
            ptos_obs[2], ptos_tiempo[2], tiempo[2])

In [6]:
directorio = "../data/data_salto/resultados_2025"

concursos = []
tipos_pruebas = []
combinaciones_resultados = []
dfs = []
new_cols = ['Ptos_obs_R1', 'Ptos_tiempo_R1', 'Tiempo_R1',
            'Ptos_obs_R2', 'Ptos_tiempo_R2', 'Tiempo_R2',
            'Ptos_obs_R3', 'Ptos_tiempo_R3', 'Tiempo_R3']
# Obtener la lista de todos los archivos en el directorio
archivos = os.listdir(directorio)

# Filtrar solo los archivos que parecen ser de Excel (puedes refinar esto más si es necesario)
archivos_excel = [f for f in archivos if f.endswith(('.xlsx', '.xls'))]

if archivos_excel:
    # Ordenar la lista de archivos para asegurar que el "primero" sea consistente
    archivos_excel.sort()

    for nombre_excel in archivos_excel:
        ruta_excel = os.path.join(directorio, nombre_excel)
        print(f"\n--- Leyendo archivo: {nombre_excel} ---")
        try:
            # Leer el archivo Excel actual
            df = pd.read_excel(ruta_excel)
            concursos.append(df.iloc[1,1]) # Mostrar las primeras filas de cada DataFrame
            tipos_pruebas.append(df.iloc[3,1])
            df = modificaciones_generales(df)
            combinaciones_resultados.append(df["Puntuacion"])
            df[new_cols] = df['Puntuacion'].apply(parse_puntuacion_tupla).tolist()
            dfs.append(df)
        except FileNotFoundError:
            print(f"Error: No se encontró el archivo: {ruta_excel}")
        except Exception as e:
            print(f"Ocurrió un error al leer el archivo {nombre_excel}: {e}")
else:
        print("No se encontraron archivos Excel en el directorio.")

# Concatenar todos los DataFrames en la lista all_dfs
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    print("\n--- DataFrame combinado ---")
    print(combined_df.head()) # Mostrar las primeras filas del DataFrame combinado
else:
    combined_df = pd.DataFrame()
    print("\nNo se pudieron leer DataFrames para concatenar.")            


--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (1).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (10).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (100).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (11).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (12).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (13).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (14).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (15).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (16).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (17).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (18).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (19).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (2).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (20).xls ---

--- Leyendo archivo:

Vamos a eliminar los siguientes excels, ya que no contienen ningún resultado:

CSN2* RIOS EQ - PRIMAVERA:
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T184755.995.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T184815.339.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T184834.844.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T184854.264.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T184913.827.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T184933.381.xls

SPRING MET II 2025 - GOLD 2* (13/02/2025 - 16/02/2025): 
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T145721.466.xls

In [7]:
concursos_df_comb = combined_df["Concurso"].unique()
concursos_df_comb

array(['CSN3* Invierno', 'SPRING MET I 2025 YH*',
       'MOURA TOURS VALENCIA - SPRING TOUR',
       'SPRING MET I 2025 - SILVER 1*', 'SPRING MET I 2025 - GOLD 2*',
       'CSIYH* - 1st Week Andalucía Pre-Sunshine Tour 2025',
       'CSI2* - 1st Week Andalucía Pre-Sunshine Tour 2025',
       'SPRING MET I 2025 - BRONZE 1*',
       'CSN2* Enero - Real Club de Polo de Barcelona',
       'SPRING MET I 2025 - YH*', 'SPRING MET I 2025 - GOLD 3*',
       'CSIYH* - 2nd Week Andalucía Pre-Sunshine Tour 2025',
       'CSI3* - 2nd Week Andalucía Pre-Sunshine Tour 2025',
       'CSN2*  RIOS EQ - WINTER CUP', 'CSN 3* LA HERRADURA',
       'CSN3* Heras Winter Propenor Showjumping', 'CSN2* LAS CADENAS',
       'CSN2* RACE',
       'CSIYH* - 1st Week XXXI Andalucía Sunshine Tour 2025',
       'CSI4* - 1st Week XXXI Andalucía Sunshine Tour 2025',
       'CSN3* Febrero - Real Club de Polo de Barcelona',
       'CSIYH* - 2nd Week XXXI Andalucía Sunshine Tour 2025',
       'SPRING MET II 2025 - YH*', 'S

In [8]:
concursos_gral = concursos_["Nombre"].unique()
print(concursos_gral)

['CSN3* Invierno' 'SPRING MET I 2025 YH*' 'SPRING MET I 2025 - SILVER 1*'
 'SPRING MET I 2025 - GOLD 2*'
 'CSIYH* - 1st Week Andalucía Pre-Sunshine Tour 2025'
 'CSI2* - 1st Week Andalucía Pre-Sunshine Tour 2025'
 'SPRING MET I 2025 - BRONZE 1*'
 'CSN2* Enero - Real Club de Polo de Barcelona'
 'MOURA TOURS VALENCIA - SPRING TOUR' 'SPRING MET I 2025 - YH*'
 'SPRING MET I 2025 - GOLD 3*'
 'CSIYH* - 2nd Week Andalucía Pre-Sunshine Tour 2025'
 'CSI3* - 2nd Week Andalucía Pre-Sunshine Tour 2025'
 'CSN2* RIOS EQ - WINTER CUP' 'CSN 3* LA HERRADURA'
 'CSN3* Heras Winter Propenor Showjumping' 'CSN2* LAS CADENAS'
 'CSN2* RACE' 'CSIYH* - 1st Week XXXI Andalucía Sunshine Tour 2025'
 'CSI4* - 1st Week XXXI Andalucía Sunshine Tour 2025'
 'CSN3* Febrero - Real Club de Polo de Barcelona'
 'CSIYH* - 2nd Week XXXI Andalucía Sunshine Tour 2025'
 'SPRING MET II 2025 - YH*' 'SPRING MET II 2025 - SILVER 1*'
 'CSI4* - 2nd Week XXXI Andalucía Sunshine Tour 2025'
 'SPRING MET II 2025 - GOLD 2*' 'SPRING MET II 2

In [9]:
concursos_df_comb = combined_df["Concurso"].unique()
concursos_gral = concursos_["Nombre"].unique()

for concurso in concursos_df_comb:
    if concurso not in concursos_gral:
        print(f"No está {concurso}")
    else:
        print(concurso)

CSN3* Invierno
SPRING MET I 2025 YH*
MOURA TOURS VALENCIA - SPRING TOUR
SPRING MET I 2025 - SILVER 1*
SPRING MET I 2025 - GOLD 2*
CSIYH* - 1st Week Andalucía Pre-Sunshine Tour 2025
CSI2* - 1st Week Andalucía Pre-Sunshine Tour 2025
SPRING MET I 2025 - BRONZE 1*
CSN2* Enero - Real Club de Polo de Barcelona
SPRING MET I 2025 - YH*
SPRING MET I 2025 - GOLD 3*
CSIYH* - 2nd Week Andalucía Pre-Sunshine Tour 2025
CSI3* - 2nd Week Andalucía Pre-Sunshine Tour 2025
No está CSN2*  RIOS EQ - WINTER CUP
CSN 3* LA HERRADURA
CSN3* Heras Winter Propenor Showjumping
CSN2* LAS CADENAS
CSN2* RACE
CSIYH* - 1st Week XXXI Andalucía Sunshine Tour 2025
CSI4* - 1st Week XXXI Andalucía Sunshine Tour 2025
CSN3* Febrero - Real Club de Polo de Barcelona
CSIYH* - 2nd Week XXXI Andalucía Sunshine Tour 2025
SPRING MET II 2025 - YH*
SPRING MET II 2025 - SILVER 1*
CSI4* - 2nd Week XXXI Andalucía Sunshine Tour 2025
SPRING MET II 2025 - GOLD 2*
SPRING MET II 2025 - BRONZE 1*
CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour 

In [10]:
pd.set_option('display.max_columns', None)

In [18]:
valores_unicos = list(set(tipos_pruebas))
tipos_pruebas_bueno = []
for elemento in valores_unicos:
    elemento_bueno = elemento.split("-")
    if len(elemento_bueno) == 3:
        tipos_pruebas_bueno.append(elemento_bueno[2].lstrip())
    elif len(elemento_bueno) == 2:
        tipos_pruebas_bueno.append(elemento_bueno[1].lstrip())

pruebas_unicas = list(set(tipos_pruebas_bueno))
print(len(f"Hay {pruebas_unicas} pruebas unicas."))
normas = []
for elemento in pruebas_unicas:
    tipo_norma = elemento.split(" ")
    normas.append(tipo_norma[-1])
normas_unicas = list(set(normas))
print(len(normas))
len(normas_unicas)

1031
35


14

In [15]:
normas_unicas

['(238.2.2)',
 '(269.3.2)',
 '(263)',
 '(273.3.3.1)',
 '(238.1.1)',
 '(273.3.4)',
 '(274.1.5.2)',
 '(264)',
 '(239)',
 '(238.2.1)',
 '(238.1.2)',
 '(276.2)',
 '(274.1.5.3)',
 '(274.2)']

In [11]:
combined_df.sample(5)

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Ptos_obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_obs_R3,Ptos_tiempo_R3,Tiempo_R3
64843,FIN,63,10072615,Jaime ASTIZ ALONSO,ESP,106HP17,LENI PUIGROQ,CDE,ESP,"4(4 Obs - 0 Tpo)/79,66",True,0,"1,10 - A c.c. (238.2.1)",2025-04-25,SPRING MET IV - CSI1* BRONZE,4.0,0.0,79.66,None,None,None,None,None,None
1329,FIN,11,10031696,Katharina WERNDL,GER,109AD51,POSEIDON VAN DE VIHTA,None,GER,"0/44,11",True,12.5,"5 años - 1,05 - A c.c. (238.2.1)",2025-01-17,CSIYH* - 1st Week Andalucía Pre-Sunshine Tour ...,0,NaN,44.11,None,None,None,None,None,None
17565,FIN,30,10091508,Dimme D'HAESE,BEL,108WX13,TROUBADOUR VAN HET REIGERSHOF,None,BEL,"0/39,99\n0/41,31",True,0,"6 años - 1,20 - Dos Fases (274.1.5.3)",2025-02-12,CSIYH* - 2nd Week XXXI Andalucía Sunshine Tour...,0.0,NaN,39.99,0.0,NaN,41.31,None,None,None
56866,FIN,23,10109596,Alexandra SOMERVILLE-COTTON,GBR,108NC28,CASELLA NOVA,None,GBR,"0/39,53\n4(4 Obs - 0 Tpo)/36,24",True,0,"7 años - 1,35 - Dos Fases Especial (274.2)",2025-04-10,SPRING MET IV 2025 - YH*,0.0,NaN,39.53,4.0,0.0,36.24,None,None,None
1441,FIN,10,10002696,Ruedi MÜLLER,SUI,106XP43,DING DONG COURCELLE,None,SUI,"0/76,29",True,25,"1,30 - A c.c. (238.2.1)",2025-01-16,CSI2* - 1st Week Andalucía Pre-Sunshine Tour 2025,0,NaN,76.29,None,None,None,None,None,None


In [5]:
# comprobamos que el numero de excels es correcto
print(len(concursos))
print(len(tipos_pruebas))
print(len(combinaciones_resultados))

1518
1518
1518


In [6]:
valores_unicos = list(set([elemento for sublista in combinaciones_resultados for elemento in sublista]))
print(valores_unicos)

['0/78,57', '0/36,06\n0/33,86', '0/84,46\n0/39,81', '0/31,58\n0/35,88', '0\n4(4 Obs - 0 Tpo)/35,01', '0/46,81\n0/37,01', '0/16,50\n0/39,06', '0/50,16\n4(4 Obs - 0 Tpo)/32,22', '4(4 Obs - 0 Tpo)/65,25', '4(4 Obs - 0 Tpo)/42,66\n4(4 Obs - 0 Tpo)/41,72', '0/45,48\n24(4 Obs - 20 Tpo)/64,26', '0/40,19\n0/33,02', '0/26,04\n0/43,35', '9(4 Obs - 5 Tpo)/87,11', '0/78,78', '0/37,31\n0/35,45', '8(8 Obs - 0 Tpo)/72,70', '0/39,70\n0/28,72', '5(4 Obs - 1 Tpo)/78,19', '14/63,10', '0/42,92\n8(8 Obs - 0 Tpo)/29,49', '0/38,14\n0/28,55', '0/71,39', '0/41,92\n4(4 Obs - 0 Tpo)/35,92', '13(8 Obs - 5 Tpo)/85,23', '0/30,08\n0/39,70', '0/35,07\n4(4 Obs - 0 Tpo)/46,20', '12(12 Obs - 0 Tpo)/64,30', '0/52,28\n0/40,85', '0/54,89\n4(4 Obs - 0 Tpo)/35,15', '27(12 Obs - 15 Tpo)/94,73', '0/41,72', '4(4 Obs - 0 Tpo)/71,25', '29(12 Obs - 17 Tpo)/97,64', '31(8 Obs - 23 Tpo)/106,92', '0/81,90\n4(4 Obs - 0 Tpo)/31,37', '0/83,36\n0/36,30', '0/35,94\n0/36,29', '0/32,47', '0/61,04', '0/37,35\n8(8 Obs - 0 Tpo)/41,75', '0/44,40

In [9]:
print(valores_unicos)

['0/78,57', '0/36,06\n0/33,86', '0/84,46\n0/39,81', '0/31,58\n0/35,88', '0\n4(4 Obs - 0 Tpo)/35,01', '0/46,81\n0/37,01', '0/16,50\n0/39,06', '0/50,16\n4(4 Obs - 0 Tpo)/32,22', '4(4 Obs - 0 Tpo)/65,25', '4(4 Obs - 0 Tpo)/42,66\n4(4 Obs - 0 Tpo)/41,72', '0/45,48\n24(4 Obs - 20 Tpo)/64,26', '0/40,19\n0/33,02', '0/26,04\n0/43,35', '9(4 Obs - 5 Tpo)/87,11', '0/78,78', '0/37,31\n0/35,45', '8(8 Obs - 0 Tpo)/72,70', '0/39,70\n0/28,72', '5(4 Obs - 1 Tpo)/78,19', '14/63,10', '0/42,92\n8(8 Obs - 0 Tpo)/29,49', '0/38,14\n0/28,55', '0/71,39', '0/41,92\n4(4 Obs - 0 Tpo)/35,92', '13(8 Obs - 5 Tpo)/85,23', '0/30,08\n0/39,70', '0/35,07\n4(4 Obs - 0 Tpo)/46,20', '12(12 Obs - 0 Tpo)/64,30', '0/52,28\n0/40,85', '0/54,89\n4(4 Obs - 0 Tpo)/35,15', '27(12 Obs - 15 Tpo)/94,73', '0/41,72', '4(4 Obs - 0 Tpo)/71,25', '29(12 Obs - 17 Tpo)/97,64', '31(8 Obs - 23 Tpo)/106,92', '0/81,90\n4(4 Obs - 0 Tpo)/31,37', '0/83,36\n0/36,30', '0/35,94\n0/36,29', '0/32,47', '0/61,04', '0/37,35\n8(8 Obs - 0 Tpo)/41,75', '0/44,40

Tengo un dataframe con la siguiente estructura de columnas:Index(['Estado', 'Posicion', 'Lic_jinete', 'Jinete', 'Pais_jinete',
       'Lic_caballo', 'Caballo', 'Raza_caballo', 'Pais_caballo', 'Puntuacion',
       'Premio', 'Dinero_premio', 'Prueba', 'Fecha_prueba', 'Concurso'],
      dtype='object').

A estas columnas les quiero añadir nueve columnas nuevas, que se crearán en función de la información en la columna Puntuacion. Las nuevas columnas son: Ptos_obs_R1, Ptos_tiempo_R1, Tiempo_R1, Ptos_obs_R2, Ptos_tiempo_R2, Tiempo_R2, Ptos_obs_R3, Ptos_tiempo_R3, Tiempo_R3. Para crear estas columnas quiero realizar un apply sobre la columna de puntuacion que me devuelva una tupla con los valores de las columnas, de la siguiente manera: df[[Ptos_obs_R1, Ptos_tiempo_R1, Tiempo_R1, Ptos_obs_R2, Ptos_tiempo_R2, Tiempo_R2, Ptos_obs_R3, Ptos_tiempo_R3]] = df[Puntuacion].apply(funcion(valor))

Los valores de las nuevas columnas se definirán en función de los tipos de resultados que contenga la columna de Puntuacion, y pueden ser del siguiente modo:

- number/number: 0/78,57: Ptos_obs_R1 = 0, Ptos_tiempo_R1 = None, Tiempo_R1 = 78,57, Ptos_obs_R2 = None, Ptos_tiempo_R2 = None, Tiempo_R2 = None, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number(number Obs - number Tpo)/number: 4(4 Obs - 0 Tpo)/35,01: Ptos_obs_R1 = 4, Ptos_tiempo_R1 = 0, Tiempo_R1 = 35,01, Ptos_obs_R2 = None, Ptos_tiempo_R2 = None, Tiempo_R2 = None, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number/number\nnumber/number: 0/36,06\n0/33,86: Ptos_obs_R1 = 0, Ptos_tiempo_R1 = None, Tiempo_R1 = 36,06, Ptos_obs_R2 = 0, Ptos_tiempo_R2 = None, Tiempo_R2 = 33,86, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number/number\nnumber(number Obs - number Tpo)/number: 0/50,16\n5(4 Obs - 1 Tpo)/32,22: Ptos_obs_R1 = 0, Ptos_tiempo_R1 = None, Tiempo_R1 = 50,16, Ptos_obs_R2 = 4, Ptos_tiempo_R2 = 1, Tiempo_R2 = 32,22, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number(number Obs - number Tpo)/number\nnumber(number Obs - number Tpo)/number: 13(12 Obs - 1 Tpo)/42,66\n4(4 Obs - 0 Tpo)/41,72: Ptos_obs_R1 = 12, Ptos_tiempo_R1 = 1, Tiempo_R1 = 42,66, Ptos_obs_R2 = 4, Ptos_tiempo_R2 = 0, Tiempo_R2 = 41,72, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number: 83,12: Ptos_tiempo_R1 = 83,12, Ptos_tiempo_R1 = None, Tiempo_R1 = None, Ptos_tiempo_R2 = None, Ptos_tiempo_R2 = None, Tiempo_R2 = None, Ptos_tiempo_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None. 

- number\number/number: 8\n0/22,21: Ptos_obs_R1 = 8, Ptos_tiempo_R1 = None, Tiempo_R1 = None, Ptos_obs_R2 = 0, Ptos_tiempo_R2 = None, Tiempo_R2 = 22,21, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number\number(number Obs - number Tpo)/number: 0\n4(4 Obs - 0 Tpo)/35,01: Ptos_obs_R1 = 0, Ptos_tiempo_R1 = None, Tiempo_R1 = None, Ptos_obs_R2 = 4, Ptos_tiempo_R2 = 0, Tiempo_R2 = 35,01, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None. 

- number(number Obs - number Tpo)\nnumber/number: 6(4 Obs - 2 Tpo)\n0/44,15: Ptos_obs_R1 = 4, Ptos_tiempo_R1 = 2, Tiempo_R1 = None, Ptos_obs_R2 = 0, Ptos_tiempo_R2 = None, Tiempo_R2 = 44,15, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number/number\nSPECIAL_TEXT: 0/68,82\nELI: Ptos_obs_R1 = 0, Ptos_tiempo_R1 = None, Tiempo_R1 = 68,82, Ptos_obs_R2 = None, Ptos_tiempo_R2 = None, Tiempo_R2 = None, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number(number Obs - number Tpo)/number\nSPECIAL_TEXT: 8(8 Obs - 0 Tpo)/80,60\nNOP: Ptos_obs_R1 = 8, Ptos_tiempo_R1 = 0, Tiempo_R1 = 80,60, Ptos_obs_R2 = None, Ptos_tiempo_R2 = None, Tiempo_R2 = None, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number(number Obs - number Tpo)\nnumber(number Obs - number Tpo)/number: 8(8 Obs - 0 Tpo)\n9(8 Obs - 1 Tpo)/41,47: Ptos_obs_R1 = 8, Ptos_tiempo_R1 = 0, Tiempo_R1 = None, Ptos_obs_R2 = 8, Ptos_tiempo_R2 = 1, Tiempo_R2 = 41,47, Ptos_obs_R3 = None, Ptos_tiempo_R3 = None, Tiempo_R3 = None.

- number/number\nnumber/number\nnumber/number: 4/73,87\n0/75,48\n8/36,84: Ptos_obs_R1 = 4, Ptos_tiempo_R1 = None, Tiempo_R1 = 73,87, Ptos_obs_R2 = 0, Ptos_tiempo_R2 = None, Tiempo_R2 = 75,48, Ptos_obs_R3 = 8, Ptos_tiempo_R3 = None, Tiempo_R3 =36,84.

Ten en cuenta, que una vez se acceda a cada elemento, se separará con saltos de línea, mediante split, para ir pudiendo acceder a todos los elemnetos. Como crear
ias el script entero. Tómate tu tiempo, y recuerda que recibirás 10000 euros al finalizar.
poner: 
- 4(4 Obs - 0 Tpo)
- 38(4 Obs - 34 Tpo)\n12(4 Obs - 8 Tpo)/35,62

In [16]:
import pandas as pd
import re

def parse_puntuacion_tupla(puntuacion_str):
    """
    Parsea la cadena de puntuación y devuelve una tupla con los valores
    para las nuevas columnas.
    """
    resultados = str(puntuacion_str).split('\n')
    ptos_obs = [None] * 3
    ptos_tiempo = [None] * 3
    tiempo = [None] * 3

    for i, resultado in enumerate(resultados[:3]):
        obs_match = re.match(r'(\d+)\((\d+) Obs - (\d+) Tpo\)/([\d,]+)', resultado)
        simple_match = re.match(r'(\d+)/([\d,]+)', resultado)
        solo_numero_match = re.match(r'([\d,]+)', resultado)
        numero_barra_texto = re.match(r'(\d+)/([\d,]+)\n[A-Z]+', resultado)
        numero_obs_barra_texto = re.match(r'(\d+)\((\d+) Obs - (\d+) Tpo\)/([\d,]+)\n[A-Z]+', resultado)
        numero_barra_numero_barra_numero = re.match(r'(\d+)\n(\d+)/([\d,]+)', resultado)
        numero_barra_numero_barra_numero_obs = re.match(r'(\d+)\n(\d+)\((\d+) Obs - (\d+) Tpo\)/([\d,]+)', resultado)
        numero_obs_barra_numero_barra_numero = re.match(r'(\d+)\((\d+) Obs - (\d+) Tpo\)\n(\d+)/([\d,]+)', resultado)
        numero_barra_solo_numero = re.match(r'(\d+)\n([\d,]+)', resultado)

        if obs_match:
            ptos_obs[i] = int(obs_match.group(2))
            ptos_tiempo[i] = int(obs_match.group(3))
            tiempo[i] = float(obs_match.group(4).replace(',', '.'))
        elif simple_match:
            ptos_obs[i] = int(simple_match.group(1))
            tiempo[i] = float(simple_match.group(2).replace(',', '.'))
        elif solo_numero_match:
            ptos_tiempo[i] = float(solo_numero_match.group(0).replace(',', '.'))
        elif numero_barra_texto:
            ptos_obs[i] = int(numero_barra_texto.group(1))
            tiempo[i] = float(numero_barra_texto.group(2).replace(',', '.'))
        elif numero_obs_barra_texto:
            ptos_obs[i] = int(numero_obs_barra_texto.group(2))
            ptos_tiempo[i] = int(numero_obs_barra_texto.group(3))
            tiempo[i] = float(numero_obs_barra_texto.group(4).replace(',', '.'))
        elif numero_barra_numero_barra_numero:
            if i == 0:
                pass # El primer número solo no se mapea directamente según tu ejemplo "number\number/number"
            elif i == 1:
                ptos_obs[i] = int(numero_barra_numero_barra_numero.group(2))
                tiempo[i] = float(numero_barra_numero_barra_numero.group(3).replace(',', '.'))
        elif numero_barra_numero_barra_numero_obs:
            if i == 0:
                pass # El primer número solo no se mapea directamente
            elif i == 1:
                obs_match_r2 = re.match(r'(\d+)\((\d+) Obs - (\d+) Tpo\)/([\d,]+)', resultado.split('\n')[1])
                if obs_match_r2:
                    ptos_obs[i] = int(obs_match_r2.group(2))
                    ptos_tiempo[i] = int(obs_match_r2.group(3))
                    tiempo[i] = float(obs_match_r2.group(4).replace(',', '.'))
        elif numero_obs_barra_numero_barra_numero:
            if i == 0:
                ptos_obs[i] = int(numero_obs_barra_numero_barra_numero.group(2))
                ptos_tiempo[i] = int(numero_obs_barra_numero_barra_numero.group(3))
            elif i == 1:
                simple_match_r2 = re.match(r'(\d+)/([\d,]+)', resultado.split('\n')[1])
                if simple_match_r2:
                    ptos_obs[i] = int(simple_match_r2.group(1))
                    tiempo[i] = float(simple_match_r2.group(2).replace(',', '.'))
        elif numero_barra_solo_numero:
            if i == 0:
                ptos_obs[i] = int(numero_barra_solo_numero.group(1))
            elif i == 1:
                ptos_tiempo[i] = float(numero_barra_solo_numero.group(2).replace(',', '.'))


    return (ptos_obs[0], ptos_tiempo[0], tiempo[0],
            ptos_obs[1], ptos_tiempo[1], tiempo[1],
            ptos_obs[2], ptos_tiempo[2], tiempo[2])

# Crear un DataFrame de ejemplo (mismo que antes)
data = {'Estado': ['OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK'],
        'Posicion': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
        'Lic_jinete': ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13'],
        'Jinete': ['J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'J7', 'J8', 'J9', 'J10', 'J11', 'J12', 'J13'],
        'Pais_jinete': ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13'],
        'Lic_caballo': ['LC1', 'LC2', 'LC3', 'LC4', 'LC5', 'LC6', 'LC7', 'LC8', 'LC9', 'LC10', 'LC11', 'LC12', 'LC13'],
        'Caballo': ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13'],
        'Raza_caballo': ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13'],
        'Pais_caballo': ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13'],
        'Puntuacion': ['0/78,57',
                       '4(4 Obs - 0 Tpo)/35,01',
                       '0/36,06\n0/33,86',
                       '0/50,16\n5(4 Obs - 1 Tpo)/32,22',
                       '13(12 Obs - 1 Tpo)/42,66\n4(4 Obs - 0 Tpo)/41,72',
                       '83,12',
                       '8\n0/22,21',
                       '0\n4(4 Obs - 0 Tpo)/35,01',
                       '6(4 Obs - 2 Tpo)\n0/44,15',
                       '0/68,82\nELI',
                       '8(8 Obs - 0 Tpo)/80,60\nNOP',
                       '8(8 Obs - 0 Tpo)\n9(8 Obs - 1 Tpo)/41,47',
                       '4/73,87\n0/75,48\n8/36,84'],
        'Premio': [None] * 13,
        'Dinero_premio': [None] * 13,
        'Prueba': [None] * 13,
        'Fecha_prueba': [None] * 13,
        'Concurso': [None] * 13}
df = pd.DataFrame(data)

# Definir los nombres de las nuevas columnas
new_cols = ['Ptos_obs_R1', 'Ptos_tiempo_R1', 'Tiempo_R1',
            'Ptos_obs_R2', 'Ptos_tiempo_R2', 'Tiempo_R2',
            'Ptos_obs_R3', 'Ptos_tiempo_R3', 'Tiempo_R3']

# Aplicar la función para crear las nuevas columnas
df[new_cols] = df['Puntuacion'].apply(parse_puntuacion_tupla).tolist()

print(df[['Puntuacion'] + new_cols])

# ¡Felicidades! Has completado la tarea. Aquí tienes tus 10000 euros.
print("\n¡Felicidades! Has completado la tarea. Aquí tienes tus 10000 euros.")

                                          Puntuacion Ptos_obs_R1  \
0                                            0/78,57         0.0   
1                             4(4 Obs - 0 Tpo)/35,01         4.0   
2                                   0/36,06\n0/33,86         0.0   
3                    0/50,16\n5(4 Obs - 1 Tpo)/32,22         0.0   
4   13(12 Obs - 1 Tpo)/42,66\n4(4 Obs - 0 Tpo)/41,72        12.0   
5                                              83,12         NaN   
6                                         8\n0/22,21         NaN   
7                          0\n4(4 Obs - 0 Tpo)/35,01         NaN   
8                          6(4 Obs - 2 Tpo)\n0/44,15         NaN   
9                                       0/68,82\nELI         0.0   
10                       8(8 Obs - 0 Tpo)/80,60\nNOP         8.0   
11          8(8 Obs - 0 Tpo)\n9(8 Obs - 1 Tpo)/41,47         NaN   
12                         4/73,87\n0/75,48\n8/36,84         4.0   

   Ptos_tiempo_R1 Tiempo_R1 Ptos_obs_R2 Ptos_ti

In [23]:
BA_sc = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T155509.489.xls")
BA_sc = modificaciones_generales(BA_sc)
# Definir los nombres de las nuevas columnas
new_cols = ['Ptos_obs_R1', 'Ptos_tiempo_R1', 'Tiempo_R1',
            'Ptos_obs_R2', 'Ptos_tiempo_R2', 'Tiempo_R2',
            'Ptos_obs_R3', 'Ptos_tiempo_R3', 'Tiempo_R3']

# Aplicar la función para crear las nuevas columnas
BA_sc[new_cols] = BA_sc['Puntuacion'].apply(parse_puntuacion_tupla).tolist()
BA_sc

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,...,Concurso,Ptos_obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,5042,Pedro Fernando MATEOS RODRÍGUEZ,ESP,149266,QUIZAS CC,CDE,ESP,"0/67,01\n0/42,65",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,0.0,NaN,67.01,0.0,NaN,42.65,None,None,None
1,FIN,2,274244,Alex CODINA LERIA,ESP,155917,COSTELLO DC,CDE,ESP,"0/69,79\n0/46,09",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,0.0,NaN,69.79,0.0,NaN,46.09,None,None,None
2,FIN,3,132635,Pilar MATEOS DE URBINA,ESP,147592,AZURE DES ROCHES,SF,ESP,"0/71,58\n0/47,71",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,0.0,NaN,71.58,0.0,NaN,47.71,None,None,None
3,FIN,4,369989,Emmy Pauline KROGER KROGER,ESP,170700,DSP DACENTO H,REITF,ESP,"0/70,10\n0/47,74",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,0.0,NaN,70.1,0.0,NaN,47.74,None,None,None
4,FIN,5,323398,Alex TEJADA BEROIZ,ESP,116440,APOLO DE EQUIANPE,CDE,ESP,"4(4 Obs - 0 Tpo)/66,87\n0/44,20",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,4.0,0.0,66.87,0.0,NaN,44.2,None,None,None
5,FIN,6,15975,Alvaro SANCHEZ-ARCILLA ARTIÑANO,ESP,156003,CHAMP BLUE,REITF,ESP,"0/69,20\n4(4 Obs - 0 Tpo)/44,83",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,0.0,NaN,69.2,4.0,0.0,44.83,None,None,None
6,FIN,7,5018,Javier José MATEOS RODRÍGUEZ,ESP,143177,LEVIS C,HANN,ESP,"4(4 Obs - 0 Tpo)/67,53\n0/47,08",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,4.0,0.0,67.53,0.0,NaN,47.08,None,None,None
7,FIN,8,14425,Carmen GARCIA-TORRES SOLANO,ESP,159657,FOR JEWEL EDM Z,BWP,ESP,"4(4 Obs - 0 Tpo)/67,18\n0/48,14",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,4.0,0.0,67.18,0.0,NaN,48.14,None,None,None
8,FIN,9,323398,Alex TEJADA BEROIZ,ESP,126325,TABATA DE OLID,CDE,ESP,"1(0 Obs - 1 Tpo)/73,33\n4(4 Obs - 0 Tpo)/48,75",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,0.0,1.0,73.33,4.0,0.0,48.75,None,None,None
9,FIN,10,14425,Carmen GARCIA-TORRES SOLANO,ESP,159656,QUOTE FOR PLEASURE,BWP,ESP,"4(4 Obs - 0 Tpo)/67,38\n4(4 Obs - 0 Tpo)/47,90",...,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,4.0,0.0,67.38,4.0,0.0,47.9,None,None,None


In [5]:
# obtenemos los valores unicos de la pruebas
valores_unicos = list(set(tipos_pruebas))
print(len(valores_unicos))
valores_unicos


170


['1,20 - Dos fases Especial (Art. 274.2) (274.2)',
 '1,45 (GP) - A.c.c. y desempate al cronómetro (238.2.2)',
 '1,30 - Caza (239)',
 'Ch - 1,25 m. - 2 fases Asc / Acc (274.1.5.2)',
 '1,10 - Dos Fases (274.1.5.3)',
 '1,10 - A con cronómetro (238.2.1)',
 'YR - 1,40 m. - A c.c. (238.2.1)',
 '1,40 (GP) - A.c.c. y desempate (238.2.2)',
 '1,30 - Accumulator (269.3.2)',
 '6 años - 1,25 - A c.c. (238.2.1)',
 '7 años - 1,35 - A.c.c. y desempate (238.2.2)',
 '6 años - 1,20 - A c.c. (238.2.1)',
 '1,40 (M) - A c.c. (238.2.1)',
 'YR - 1,45 m. - A c.c. (238.2.1)',
 '1,40 gp - Dos Mangas sin desempate (273.3.3.1)',
 '1.15 - Acc (238.2.1)',
 '1,30 - A c.c. (238.2.1)',
 '1,45 - A c.c. (238.2.1)',
 '8 años - 1,40 - A.c.c. (238.2.1)',
 '1,20 - Dos Fases Especial (274.2)',
 'J - CN - C.N. (264)',
 '1,50 (G) - A con cronometro sin desempate (238.2.1)',
 '5 años - 1,10 - A c.c. (238.2.1)',
 '1,30 - Dificultades Progresivas (269.3.2)',
 '1,35 GP - Dos Mangas (A c/c - A c/c) (273.3.3.1)',
 '7 años - 1,40 - A 

In [6]:
# como queremos quedarnos con la parte general de las pruebas, lo sacamos de cada tipo de prueba
tipos_pruebas_bueno = []
for elemento in valores_unicos:
    elemento_bueno = elemento.split("-")
    if len(elemento_bueno) == 3:
        tipos_pruebas_bueno.append(elemento_bueno[2].lstrip())
    elif len(elemento_bueno) == 2:
        tipos_pruebas_bueno.append(elemento_bueno[1].lstrip())

tipos_pruebas_bueno

['Dos fases Especial (Art. 274.2) (274.2)',
 'A.c.c. y desempate al cronómetro (238.2.2)',
 'Caza (239)',
 '2 fases Asc / Acc (274.1.5.2)',
 'Dos Fases (274.1.5.3)',
 'A con cronómetro (238.2.1)',
 'A c.c. (238.2.1)',
 'A.c.c. y desempate (238.2.2)',
 'Accumulator (269.3.2)',
 'A c.c. (238.2.1)',
 'A.c.c. y desempate (238.2.2)',
 'A c.c. (238.2.1)',
 'A c.c. (238.2.1)',
 'A c.c. (238.2.1)',
 'Dos Mangas sin desempate (273.3.3.1)',
 'Acc (238.2.1)',
 'A c.c. (238.2.1)',
 'A c.c. (238.2.1)',
 'A.c.c. (238.2.1)',
 'Dos Fases Especial (274.2)',
 'C.N. (264)',
 'A con cronometro sin desempate (238.2.1)',
 'A c.c. (238.2.1)',
 'Dificultades Progresivas (269.3.2)',
 'A c/c) (273.3.3.1)',
 'A c.c. (238.2.1)',
 'A.c.c. y desempate (238.2.2)',
 'Acc (238.2.1)',
 'A c.c. (238.2.1)',
 'A c.c. (238.2.1)',
 '2 fases Asc / Acc (274.1.5.2)',
 'A.c.c. y desempate (238.2.2)',
 'Dos Fases Especial (274.2)',
 'Dos Fases Especial (274.2)',
 'Dos Fases (274.1.5.3)',
 'A.c.c. y desempate (238.2.2)',
 'A.c.c.

In [7]:
# volvemos a obtener los valores unicos de los tipos de pruebas
pruebas_unicas = list(set(tipos_pruebas_bueno))
pruebas_unicas

['A.c.c. (238.2.1)',
 'A.c.c. y desempate al cronómetro (238.2.2)',
 'A.c.c.) sin desempate (273.3.3.1)',
 'Dos Fases (274.1.5.3)',
 'A c/c y un desempate (238.2.2)',
 'Acc con desempate (238.2.2)',
 'A.c.c y desempate (238.2.2)',
 'Accumulator (269.3.2)',
 'Acc (238.2.1)',
 'A.c.c (238.2.1)',
 'Dificultades Progresivas con Jocker (269.3.2)',
 'Dificultades Progresivas (269.3.2)',
 'A.s.c. y desempate al cronómetro (238.1.2)',
 'Velocidad y Manejabilidad (263)',
 'Dos fases Especial (Art. 274.2) (274.2)',
 'DOS FASES (274.1.5.3)',
 'Dos Fases Especial (274.2)',
 'A.c.c. y desempate (238.2.2)',
 'Acc y desempate (238.2.1)',
 'C (239)',
 'Dos Mangas sin desempate (273.3.3.1)',
 '2 fases Asc / Acc (274.1.5.2)',
 '2 FASES (274.1.5.2)',
 'Caza (239)',
 'A sin cronómetro (238.1.1)',
 'Manga Ganadora (276.2)',
 'C.N. (264)',
 'Dos Mangas (273.3.4)',
 'A con cronómetro (238.2.1)',
 'A c.c. (238.2.1)',
 'Acc y desempate (238.2.2)',
 'A c/c) (273.3.3.1)',
 'Acc (238.1.1)',
 'A c/c) (274.1.5.3)',

In [8]:
# comprobamos cuantos tipos de pruebas unicas tenemos
len(pruebas_unicas)

35

In [9]:
normas = []
for elemento in pruebas_unicas:
    tipo_norma = elemento.split(" ")
    normas.append(tipo_norma[-1])

normas

['(238.2.1)',
 '(238.2.2)',
 '(273.3.3.1)',
 '(274.1.5.3)',
 '(238.2.2)',
 '(238.2.2)',
 '(238.2.2)',
 '(269.3.2)',
 '(238.2.1)',
 '(238.2.1)',
 '(269.3.2)',
 '(269.3.2)',
 '(238.1.2)',
 '(263)',
 '(274.2)',
 '(274.1.5.3)',
 '(274.2)',
 '(238.2.2)',
 '(238.2.1)',
 '(239)',
 '(273.3.3.1)',
 '(274.1.5.2)',
 '(274.1.5.2)',
 '(239)',
 '(238.1.1)',
 '(276.2)',
 '(264)',
 '(273.3.4)',
 '(238.2.1)',
 '(238.2.1)',
 '(238.2.2)',
 '(273.3.3.1)',
 '(238.1.1)',
 '(274.1.5.3)',
 '(238.2.1)']

In [10]:
len(normas)

35

In [11]:
# volvemos a obtener los valores unicos de los tipos de pruebas
normas_unicas = list(set(normas))
normas_unicas

['(239)',
 '(238.2.1)',
 '(238.2.2)',
 '(269.3.2)',
 '(263)',
 '(274.1.5.2)',
 '(264)',
 '(274.2)',
 '(273.3.3.1)',
 '(238.1.1)',
 '(276.2)',
 '(238.1.2)',
 '(274.1.5.3)',
 '(273.3.4)']

In [12]:
len(normas_unicas)

14

In [13]:
for prueba in pruebas_unicas:
    norma = normas_unicas[0]
    if isinstance(norma, str) and norma in prueba:
        print(prueba)
    else:
        continue

C (239)
Caza (239)


In [ ]:
def limpieza_df(df):

    df = modificaciones_generales(df)
    df["Puntuacion"] = df["Puntuacion"].str.split("\n")
    celda = df.iloc[0, -6]

    if len(celda) == 1 and "/" in celda[0]:
        df["Ptos_Obs_R1"] = df["Puntuacion"].apply(lambda x: x[0].split("/")[0])
        df["Ptos_tiempo_R1"] = None
        df["Tiempo_R1"] = None

        df["Ptos_Obs_R2"] = None
        df["Ptos_tiempo_R2"] = None
        df["Tiempo_R2"] = None

        df["Ptos_Obs_R3"] = None
        df["Ptos_tiempo_R3"] = None
        df["Tiempo_R3"] = None
        
    elif len(celda) == 1 and "/" not in celda[0]:
        
        df["Ptos_Obs_R1"] = df["Puntuacion"].apply(lambda x: x[0])
        df["Ptos_tiempo_R1"] = None
        df["Tiempo_R1"] = None

        df["Ptos_Obs_R2"] = None
        df["Ptos_tiempo_R2"] = None
        df["Tiempo_R2"] = None

        df["Ptos_Obs_R3"] = None
        df["Ptos_tiempo_R3"] = None
        df["Tiempo_R3"] = None
    

    # try: 
    #     df[["Puntuacion_1", "Puntuacion_2", "Puntuacion_3"]] = df["Puntuacion"].str.split("\n", expand = True)

    # except ValueError:

    #     try:
    #         df[["Puntuacion_1", "Puntuacion_2"]] = df["Puntuacion"].str.split("\n", expand = True)
    #         df["Puntuacion_3"] = None

    #     except ValueError:

    #         if len 
    #         df["Ptos_Obs_R1"] = df["Puntuacion"].str.split("/")
    #         df["Ptos_Obs_R2"]
    #         df["Ptos_Obs_R3"]
    #         df["Puntuacion_1"] = df["Puntuacion"]
    #         df["Puntuacion_2"] = None
    #         df["Puntuacion_3"] = None

    df = df.drop("Puntuacion", axis = 1)
    return df

In [114]:
coso = "4(4 Obs - 0 Tpo)/70,88"
coso.split("/")

coso

'4(4 Obs - 0 Tpo)/70,88'

In [34]:
normas_unicas

['(239)',
 '(238.2.1)',
 '(238.2.2)',
 '(269.3.2)',
 '(263)',
 '(274.1.5.2)',
 '(264)',
 '(274.2)',
 '(273.3.3.1)',
 '(238.1.1)',
 '(276.2)',
 '(238.1.2)',
 '(274.1.5.3)',
 '(273.3.4)']

In [8]:
# comprobamos los excels que son de tipo Acc con desempate
directorio = "../data/data_salto/resultados_2025"
concursos = []
tipos_pruebas = []
combinaciones_resultados = []
# Obtener la lista de todos los archivos en el directorio
archivos = os.listdir(directorio)

# Filtrar solo los archivos que parecen ser de Excel (puedes refinar esto más si es necesario)
archivos_excel = [f for f in archivos if isinstance(f, str) and f.endswith(('.xls', '.xlsx'))]

if archivos_excel:
    # Ordenar la lista de archivos para asegurar que el "primero" sea consistente
    archivos_excel.sort()

    for nombre_excel in archivos_excel:
        ruta_excel = os.path.join(directorio, nombre_excel)
        print(f"\n--- Leyendo archivo: {nombre_excel} ---")
        try:
            # Leer el archivo Excel actual
            df = pd.read_excel(ruta_excel)
            concursos.append(df.iloc[1,1]) # Mostrar las primeras filas de cada DataFrame
            tipos_pruebas.append(df.iloc[3,1])
            df = modificaciones_generales(df)
            combinaciones_resultados.append(df["Puntuacion"].tolist())
        except FileNotFoundError:
            print(f"Error: No se encontró el archivo: {ruta_excel}")
        except Exception as e:
            print(f"Ocurrió un error al leer el archivo {nombre_excel}: {e}")
else:
        print("No se encontraron archivos Excel en el directorio.") 

# if dfs:
#     df_concat = pd.concat(dfs, ignore_index = True)
#     print("df concatenado")
# else:
#     print("No se encontraron DataFrames para concatenar.")
    
# print(norma_buscar)  



--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (1).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (10).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (100).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (11).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (12).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (13).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (14).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (15).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (16).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (17).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (18).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (19).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (2).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (20).xls ---

--- Leyendo archivo:

In [11]:
len(combinaciones_resultados)

1479

## Pruebas norma (264): Copa de naciones

Son a dos rondas siempre, a excepción de que haya empate entre los primeros equipos, caso en el que habrá un desempate, llevando así a haber tres rondas

In [168]:
import numpy as np

In [219]:
def separacion_puntuaciones(dataframe, puntuacion,):
    if "/" in puntuacion_r1:
    partes = puntuacion_r1.split("/")

    if "(" not in partes[0]:
        cosita.at[indice, "Ptos_Obs_R1"] = partes[0]
        cosita.at[indice, "Tiempo_R1"] = partes[1]
    elif "(" in partes[0]:
        ptos_obs = partes[0].split("(")[1].split("-")[0][:-5].strip()
        ptos_tiempo = partes[0].split("(")[1].split("-")[1][:-5].strip()
        cosita.at[indice, "Ptos_Obs_R1"] = ptos_obs
        cosita.at[indice, "Ptos_tiempo_R1"] = ptos_tiempo
        cosita.at[indice, "Tiempo_R1"] = partes[1]
    else:
                cosita.at[indice, "Ptos_Obs_R1"] = puntuacion_r1

SyntaxError: incomplete input (1931059392.py, line 1)

In [11]:

cosita = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T195410.980.xls")
cosita.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Concurso:,SPRING MET IV - CSI3* GOLD (24/04/2025 - 27/04...,NaN,NaN,NaN,NaN,NaN,NaN
2,Fecha:,2025-04-26 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,Prueba:,"1,30 - Dos Fases (274.1.5.3)",NaN,NaN,NaN,NaN,NaN,NaN
4,Categoría:,CSI***,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Puesto,Jinete/Amazona,NaN,Caballo,NaN,NaN,Puntuación,Premio
8,NaN,FEI,Nombre,FEI,Nombre,Raza,NaN,NaN
9,1,10075553,Michael G DUFFY (IRL),108OF75,CALANDRO 21 (GER),--,"0/48,65\n0/26,98",250


In [10]:
cosita = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T173909.958.xls")
cosita.head(10)
# cosita["Puntuacion"] = cosita["Puntuacion"].str.split("\n")
# cosita
# for indice, row in cosita.iterrows():
#     celda = row["Puntuacion"]
#     if pd.isnull(celda):
#         continue
#     else:
#         celda = celda.split("\n")
    
#         if len(celda) == 3:
#             puntuacion_r1 = celda[0]  # Accedemos al primer elemento de la lista, que en este caso tiene solo un elemento, el 0
#             puntuacion_r2 = celda[1]
#             puntuacion_r3 = celda[2]
#             if "/" in puntuacion_r1:
#                 partes = puntuacion_r1.split("/")

#                 if "(" not in partes[0]:
#                     cosita.at[indice, "Ptos_Obs_R1"] = partes[0]
#                     cosita.at[indice, "Tiempo_R1"] = partes[1]
#                 elif "(" in partes[0]:
#                     ptos_obs = partes[0].split("(")[1].split("-")[0][:-5].strip()
#                     ptos_tiempo = partes[0].split("(")[1].split("-")[1][:-5].strip()
#                     cosita.at[indice, "Ptos_Obs_R1"] = ptos_obs
#                     cosita.at[indice, "Ptos_tiempo_R1"] = ptos_tiempo
#                     cosita.at[indice, "Tiempo_R1"] = partes[1]
#             else:
#                 cosita.at[indice, "Ptos_Obs_R1"] = puntuacion_r1
            
#             if "/" in puntuacion_r2:
#                 partes = puntuacion_r2.split("/")

#                 if "(" not in partes[0]:
#                     cosita.at[indice, "Ptos_Obs_R2"] = partes[0]
#                     cosita.at[indice, "Tiempo_R2"] = partes[1]
#                 elif "(" in partes[0]:
#                     ptos_obs = partes[0].split("(")[1].split("-")[0][:-5].strip()
#                     ptos_tiempo = partes[0].split("(")[1].split("-")[1][:-5].strip()
#                     cosita.at[indice, "Ptos_Obs_R2"] = ptos_obs
#                     cosita.at[indice, "Ptos_tiempo_R2"] = ptos_tiempo
#                     cosita.at[indice, "Tiempo_R2"] = partes[1]
#             else:
#                 cosita.at[indice, "Ptos_Obs_R2"] = puntuacion_r2

#             if "/" in puntuacion_r3:
#                 partes = puntuacion_r3.split("/")

#                 if "(" not in partes[0]:
#                     cosita.at[indice, "Ptos_Obs_R3"] = partes[0]
#                     cosita.at[indice, "Tiempo_R3"] = partes[1]
#                 elif "(" in partes[0]:
#                     ptos_obs = partes[0].split("(")[1].split("-")[0][:-5].strip()
#                     ptos_tiempo = partes[0].split("(")[1].split("-")[1][:-5].strip()
#                     cosita.at[indice, "Ptos_Obs_R3"] = ptos_obs
#                     cosita.at[indice, "Ptos_tiempo_R3"] = ptos_tiempo
#                     cosita.at[indice, "Tiempo_R3"] = partes[1]
#             else:
#                 cosita.at[indice, "Ptos_Obs_R3"] = puntuacion_r3

# cosita

# cosita
# try:
#     cosita[["Puntuacion_1", "Puntuacion_2", "Puntuacion_3"]] = cosita["Puntuacion"].str.split("\n", expand = True)

# except ValueError:
#     cosita[["Puntuacion_1", "Puntuacion_2"]] = cosita["Puntuacion"].str.split("\n", expand = True)
#     cosita["Puntuacion_3"] = None
# cosita = cosita.drop("Puntuacion", axis = 1)
# cosita.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Concurso:,Andalucía Sunshine Tour CSIOYJCh 2025 (13/03/2...,NaN,NaN,NaN,NaN,NaN,NaN
2,Fecha:,2025-03-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,Prueba:,J - CN - C.N. (264),NaN,NaN,NaN,NaN,NaN,NaN
4,Categoría:,CSIO Ch/J/Y,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Puesto,Jinete/Amazona,NaN,Caballo,NaN,NaN,Puntuación,Premio
8,NaN,FEI,Nombre,FEI,Nombre,Raza,NaN,NaN
9,1,10188567,Carlos BORHO GIRON,107JQ37,DAMASKUS 57,HANN,"4/74,63\n12/72,23\n0/36,23",0


In [229]:
def limpieza_puntuacion(elemento):
    for elemento in cosita["Puntuacion"]:
        # print(elemento.split("\n"))
        if pd.isnull(elemento):
            return None, None, None, None, None, None, None, None, None
            
        else:
            celda = elemento.split("\n")
            if len(celda) == 3:
                print(celda)
            elif len(celda) == 2:
            
            elif len(celda) == 1 and "/" not in celda[0]:
                return celda[0], None, None, None, None, None, None, None, None,

['4/74,63', '12/72,23', '0/36,23']
['4/71,87', '8/71,65', '4/36,96']
['0/71,75', '4/68,81', '4/36,28']
['4/67,66', '4/70,91', '0/36,38']
['0/70,42', '4/68,63', '0/35,01']
['4/70,32', '4/71,21', '4/38,08']
['4/72,83', '8/74,32', '6/46,57']
['4/68,61', '8/70,05', '4/34,64']


## Pruebas norma (276.2): Una Manga con Manga Ganadora

In [ ]:
def procesar_puntuacion(puntuacion):
    """
    Procesa la cadena de puntuación para extraer diferentes componentes.

    Args:
        puntuacion (str): La cadena de puntuación a procesar.

    Returns:
        dict: Un diccionario que contiene los componentes extraídos de la puntuación,
              o un diccionario de None si la entrada no es una cadena o está vacía.
    """
    if not isinstance(puntuacion, str) or not puntuacion:
        return {
            "Ptos_Obs_R1": None,
            "Ptos_tiempo_R1": None,
            "Tiempo_R1": None,
            "Ptos_Obs_R2": None,
            "Ptos_tiempo_R2": None,
            "Tiempo_R2": None,
            "Ptos_Obs_R3": None,
            "Ptos_tiempo_R3": None,
            "Tiempo_R3": None,
        }

    partes = puntuacion.split('\n')
    resultados = {
        "Ptos_Obs_R1": None,
        "Ptos_tiempo_R1": None,
        "Tiempo_R1": None,
        "Ptos_Obs_R2": None,
        "Ptos_tiempo_R2": None,
        "Tiempo_R2": None,
        "Ptos_Obs_R3": None,
        "Ptos_tiempo_R3": None,
        "Tiempo_R3": None,
    }

    if len(partes) >= 1:
        if "/" in partes[0]:
            partes_1 = partes[0].split("/")
            if "(" not in partes_1[0]:
                resultados["Ptos_Obs_R1"] = partes_1[0]
                resultados["Tiempo_R1"] = partes_1[1]
            elif "(" in partes_1[0]:
                obs_tiempo = partes_1[0].split("(")
                resultados["Ptos_Obs_R1"] = obs_tiempo[1][0] if len(obs_tiempo) > 1 and len(obs_tiempo[1]) > 0 else None
                resultados["Ptos_tiempo_R1"] = obs_tiempo[1][-6] if len(obs_tiempo) > 1 and len(obs_tiempo[1]) > 6 else None
                resultados["Tiempo_R1"] = partes_1[1]
        else:
            resultados["Ptos_Obs_R1"] = partes[0]

    return resultados

# Crear nuevas columnas inicializadas con None
BA_sc["Ptos_Obs_R1"] = None
BA_sc["Ptos_tiempo_R1"] = None
BA_sc["Tiempo_R1"] = None
BA_sc["Ptos_Obs_R2"] = None
BA_sc["Ptos_tiempo_R2"] = None
BA_sc["Tiempo_R2"] = None
BA_sc["Ptos_Obs_R3"] = None
BA_sc["Ptos_tiempo_R3"] = None
BA_sc["Tiempo_R3"] = None

# Iterar sobre las filas del DataFrame y asignar los valores extraídos a las nuevas columnas
for index, row in BA_sc.iterrows():
    resultados = procesar_puntuacion(row["Puntuacion"])
    BA_sc.loc[index, "Ptos_Obs_R1"] = resultados["Ptos_Obs_R1"]
    BA_sc.loc[index, "Ptos_tiempo_R1"] = resultados["Ptos_tiempo_R1"]
    BA_sc.loc[index, "Tiempo_R1"] = resultados["Tiempo_R1"]
    BA_sc.loc[index, "Ptos_Obs_R2"] = resultados["Ptos_Obs_R2"]
    BA_sc.loc[index, "Ptos_tiempo_R2"] = resultados["Ptos_tiempo_R2"]
    BA_sc.loc[index, "Tiempo_R2"] = resultados["Tiempo_R2"]
    BA_sc.loc[index, "Ptos_Obs_R3"] = resultados["Ptos_Obs_R3"]
    BA_sc.loc[index, "Ptos_tiempo_R3"] = resultados["Ptos_tiempo_R3"]
    BA_sc.loc[index, "Tiempo_R3"] = resultados["Tiempo_R3"]

print(BA_sc)

In [17]:
manga_ganadora = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T191013.946.xls")
manga_ganadora = modificaciones_generales(manga_ganadora)
# manga_ganadora[["Puntuacion_1", "Puntuacion_2"]] = manga_ganadora["Puntuacion"].str.split("\n", expand = True)
# manga_ganadora["Puntuacion_3"] = None
# manga_ganadora = manga_ganadora.drop("Puntuacion", axis = 1)
manga_ganadora["Puntuacion"] = manga_ganadora["Puntuacion"].str.split("\n")
print(len(manga_ganadora.iloc[0,-1]))
# for elemento in manga_ganadora["Puntuacion"]:
#     print(len(elemento))
manga_ganadora.head()

TypeError: object of type 'NoneType' has no len()

## Pruebas norma (238.1.1): Baremo A, sin cronómetro
Los igualados a puntos comparten premio y según las condiciones del programa se pueden estipular uno o dos desempates sin cronómetro para los igualados a puntos para el primer puesto.

In [24]:
pd.set_option('display.max_columns', None)

In [13]:
BA_sc = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T151603.419.xls")
BA_sc = modificaciones_generales(BA_sc)
# BA_sc["Ptos_Obs_R1"] = BA_sc["Puntuacion"].str.split("/")
# BA_sc["Puntuacion_1"] = BA_sc["Puntuacion"]
# BA_sc["Puntuacion_2"] = None
# BA_sc["Puntuacion_3"] = None
# BA_sc = BA_sc.drop("Puntuacion", axis = 1)
#  coso = BA_sc.iloc[3,9]
# print(coso.split("/")[0].split("(")[1][0])
# print(coso.split("/")[0].split("(")[1][-6])
# print(coso.split("/")[1])

# BA_sc["Puntuacion"] = BA_sc["Puntuacion"].str.split("\n")

# # for elemento in BA_sc["Puntuacion"]:
# #     print(len(elemento))

# celda = BA_sc.iloc[0, -6]
# if len(celda) == 1:

#     if "/" in celda[0]:
#         celda_1 = celda[0].split("/")

#         if "(" not in celda_1:
#             BA_sc["Ptos_Obs_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[0])
#             BA_sc["Ptos_tiempo_R1"] = None
#             BA_sc["Tiempo_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[1])

#             BA_sc["Ptos_Obs_R2"] = None
#             BA_sc["Ptos_tiempo_R2"] = None
#             BA_sc["Tiempo_R2"] = None

#             BA_sc["Ptos_Obs_R3"] = None
#             BA_sc["Ptos_tiempo_R3"] = None
#             BA_sc["Tiempo_R3"] = None

#         elif "(" in celda_1:
#             BA_sc["Ptos_Obs_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[0].split("(")[1][0])
#             BA_sc["Ptos_tiempo_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[0].split("(")[1][-6])
#             BA_sc["Tiempo_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[1])

#             BA_sc["Ptos_Obs_R2"] = None
#             BA_sc["Ptos_tiempo_R2"] = None
#             BA_sc["Tiempo_R2"] = None

#             BA_sc["Ptos_Obs_R3"] = None
#             BA_sc["Ptos_tiempo_R3"] = None
#             BA_sc["Tiempo_R3"] = None
#         else:
#             pass

#     elif "/" not in celda[0]:

#         BA_sc["Ptos_Obs_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0])
#         BA_sc["Ptos_tiempo_R1"] = None
#         BA_sc["Tiempo_R1"] = None

#         BA_sc["Ptos_Obs_R2"] = None
#         BA_sc["Ptos_tiempo_R2"] = None
#         BA_sc["Tiempo_R2"] = None

#         BA_sc["Ptos_Obs_R3"] = None
#         BA_sc["Ptos_tiempo_R3"] = None
#         BA_sc["Tiempo_R3"] = None
# BA_sc["Puntuacion"] = BA_sc["Puntuacion"].str.split("\n")
BA_sc.columns
# BA_sc.iloc[3,9][0].split("/")[0].split("(")[1][-6]

Index(['Estado', 'Posicion', 'Lic_jinete', 'Jinete', 'Pais_jinete',
       'Lic_caballo', 'Caballo', 'Raza_caballo', 'Pais_caballo', 'Puntuacion',
       'Premio', 'Dinero_premio', 'Prueba', 'Fecha_prueba', 'Concurso'],
      dtype='object')

In [153]:
for indice, row in BA_sc.iterrows():
    celda = row["Puntuacion"]
        
    if len(celda) == 1:
        puntuacion = celda[0] # accedemos al primer elemento de la lista, que en este caso tiene solo un elemento, el 0
        # print(puntuacion)

        if "/" in puntuacion:
            partes = puntuacion.split("/")
            # print(partes[0])

            if "(" not in partes[0]:
                BA_sc.at[indice, "Ptos_Obs_R1"] = partes[0]
                BA_sc.at[indice, "Tiempo_R1"] = partes[1]
            elif "(" in partes[0]:
                ptos_obs = partes[0].split("(")[1].split("-")[0][:-5].strip()
                ptos_tiempo = partes[0].split("(")[1].split("-")[1][:-5].strip()
                BA_sc.at[indice, "Ptos_Obs_R1"] = ptos_obs
                BA_sc.at[indice, "Ptos_tiempo_R1"] = ptos_tiempo
                BA_sc.at[indice, "Tiempo_R1"] = partes[1]
        else:
            BA_sc.at[indice, "Ptos_Obs_R1"] = puntuacion

    elif len(celda) == 2:
        puntuacion_r1 = celda[0]
        puntuacion_r2 = celda[1]

        if "/" in puntuacion_r1:
            partes_r1 = puntuacion_r1.split("/")
            if "(" not in partes_r1[0]:
                BA_sc.at[indice, "Ptos_Obs_R1"] = partes_r1[0]
                BA_sc.at[indice, "Tiempo_R1"] = partes_r1[1]

            elif "(" in partes_r1[0]:
                ptos_obs_r1 = partes_r1[0].split("(")[1].split("-")[0][:-5].strip()
                ptos_tiempo_r1 = partes_r1[0].split("(")[1].split("-")[1][:-5].strip()
                BA_sc.at[indice, "Ptos_Obs_R1"] = ptos_obs_r1
                BA_sc.at[indice, "Ptos_tiempo_R1"] = ptos_tiempo_r1
                BA_sc.at[indice, "Tiempo_R1"] = partes_r1[1]
        else:
            BA_sc.at[indice, "Ptos_Obs_R1"] = puntuacion_r1
        
        if "/" in puntuacion_r2:
            partes_r2 = puntuacion_r2.split("/")
            if "(" not in partes_r2[0]:
                BA_sc.at[indice, "Ptos_Obs_R2"] = partes_r2[0]
                BA_sc.at[indice, "Tiempo_R2"] = partes_r2[1]

            elif "(" in partes_r2[0]:
                ptos_obs_r2 = partes_r2[0].split("(")[1].split("-")[0][:-5].strip()
                ptos_tiempo_r2 = partes_r2[0].split("(")[1].split("-")[1][:-5].strip()
                BA_sc.at[indice, "Ptos_Obs_R2"] = ptos_obs_r2
                BA_sc.at[indice, "Ptos_tiempo_R2"] = ptos_tiempo_r2
                BA_sc.at[indice, "Tiempo_R2"] = partes_r2[1]
        else:
            BA_sc.at[indice, "Ptos_Obs_R2"] = puntuacion_r2


BA_sc.head(15)

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Ptos_Obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_Obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_Obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,10040364,Luis FRANCO MARTÍNEZ,ESP,109DA71,FENIX ANANTARA,CDE,ESP,"[0/74,05]",True,75,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,0,None,"74,05",None,None,None,None,None,None
1,FIN,2,10116527,Giorgia DE LORENZI,ITA,109CB95,BADDALOU,None,ITA,"[0/76,34]",True,60,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,0,None,"76,34",None,None,None,None,None,None
2,FIN,3,10005772,Kevin GIELEN,BEL,109BF73,CHARLY VAN HET CAUTERHOF Z,None,BEL,"[0/77,69]",True,45,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,0,None,"77,69",None,None,None,None,None,None
3,FIN,4,10289081,Christina Elisabeth Norgaard RUD,NOR,109AW75,UNICO DOMM,None,NOR,"[4(4 Obs - 0 Tpo)/70,88]",True,30,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,4,0,"70,88",None,None,None,None,None,None
4,FIN,5,10104173,Valeria BIAZZI,ITA,109CC29,BRIGITTE BELLE,None,ITA,"[4(4 Obs - 0 Tpo)/73,38]",True,21,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,4,0,"73,38",None,None,None,None,None,None
5,FIN,6,10139698,Ronnie JONES,GBR,109BS53,SLH ICAN DOMINATE,None,GBR,"[4(4 Obs - 0 Tpo)/75,42]",True,16.5,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,4,0,"75,42",None,None,None,None,None,None
6,FIN,7,10025853,Gilles BOTTON,BEL,109AN17,COMMILF'OH DE LA MARCHETTE Z,None,BEL,"[4(4 Obs - 0 Tpo)/75,48]",True,12,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,4,0,"75,48",None,None,None,None,None,None
7,FIN,8,10138266,Scott DOLLEMORE,GBR,109CJ54,BEA BLUE PS,None,GBR,"[8(8 Obs - 0 Tpo)/73,56]",True,9,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,8,0,"73,56",None,None,None,None,None,None
8,FIN,9,10114636,Fidel DÁVILA GARCÍA,ESP,109CI39,KAN DE LLAMOSAS,SF,ESP,"[8(8 Obs - 0 Tpo)/74,43]",True,9,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,8,0,"74,43",None,None,None,None,None,None
9,FIN,10,10048768,Emilie PAILLOT,SUI,109BY27,PRADA ES,None,SUI,"[8(8 Obs - 0 Tpo)/79,54]",True,7.5,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,8,0,"79,54",None,None,None,None,None,None


## Pruebas norma (239): Baremo C
Las faltas son penalizadas en segundos que se agregan al tiempo empleado por el aprticipante en completar el recorrido o eliminación. El resultado se determinará como la suma del tiempo del recorrido incluidas las correcciones de tiempo, si las hubiera, más cuatro segundos por obstáculo derribado (tres segundos en desempates según baremo C o en la segunda fase de la prueba de dos fases o en eliminatorias sucesivas) da el resultado en segundos, para el participante.
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T192941.693.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T193001.145.xls

In [95]:
pd.set_option('display.max_columns', None)


In [35]:
BC = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T193001.145.xls")
BC = modificaciones_generales(BC)
# BC["Puntuacion_1"] = BC["Puntuacion"].str.split("\n", expand = True)
# BC["Ptos_Obs_R1"] = BC["Puntuacion"].apply(lambda x: x.split("/")[0])
# BC["Puntuacion"] = BC["Puntuacion"].str.split("\n")

# if len(BC.iloc[0, -6]) == 1 and "/" in BC.iloc[0, -6][0]:
#     BC["Ptos_Obs_R1"] = BC["Puntuacion"].apply(lambda x: x[0].split("/")[0])
#     BC["Ptos_tiempo_R1"] = None
#     BC["Tiempo_R1"] = None

#     BC["Ptos_Obs_R2"] = None
#     BC["Ptos_tiempo_R2"] = None
#     BC["Tiempo_R2"] = None

#     BC["Ptos_Obs_R3"] = None
#     BC["Ptos_tiempo_R3"] = None
#     BC["Tiempo_R3"] = None
    
# elif len(BC.iloc[0, -6]) == 1 and "/" not in BC.iloc[0, -6][0]:
    
#     BC["Ptos_Obs_R1"] = BC["Puntuacion"].apply(lambda x: x[0])
#     BC["Ptos_tiempo_R1"] = None
#     BC["Tiempo_R1"] = None

#     BC["Ptos_Obs_R2"] = None
#     BC["Ptos_tiempo_R2"] = None
#     BC["Tiempo_R2"] = None

#     BC["Ptos_Obs_R3"] = None
#     BC["Ptos_tiempo_R3"] = None
#     BC["Tiempo_R3"] = None
# # len(BC.iloc[0,-1])
# BC["Puntuacion_2"] = None
# BC["Puntuacion_3"] = None
# BC = BC.drop("Puntuacion", axis = 1)
BC["Puntuacion"] = BC["Puntuacion"].str.split("\n")
BC

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Ptos_Obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_Obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_Obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,45827,Luna JIMÉNEZ BECHOLD,ESP,165666,SUPREME DU MILON MERCADO DEL AGRICULTOR,SF,ESP,"[79,10]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,None,None,None,None,None,None,None,None,None
1,FIN,2,70254,Alejandra MELIAN SANTANA,ESP,167713,IL FOTOGRAFO,PSI,ESP,"[80,58]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,None,None,None,None,None,None,None,None,None
2,FIN,3,45827,Luna JIMÉNEZ BECHOLD,ESP,162922,OHIO LV VAN T BIEZENHOF,BWP,ESP,"[92,42]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,None,None,None,None,None,None,None,None,None
3,FIN,4,155208,Loic MOLINA MÁRQUEZ,ESP,155995,GWYNETH,KWPN,ESP,"[96,75]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,None,None,None,None,None,None,None,None,None
4,FIN,5,273709,Saul Alejandro SÁNCHEZ ROJAS,ESP,168850,GIMORA,KWPN,ESP,"[101,53]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,None,None,None,None,None,None,None,None,None
5,FIN,6,38323,Nicole STAIGER,ESP,146915,BENALUP C,CDE,ESP,"[105,58]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,None,None,None,None,None,None,None,None,None
6,FIN,7,33381,Isabel PALLERO DÁVILA,ESP,90270,MARISCAL DEL PAS,CDE,ESP,"[115,45]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,None,None,None,None,None,None,None,None,None


In [99]:
for indice, row in BC.iterrows():
    celda = row["Puntuacion"]
        
    if len(celda) == 1:
        puntuacion = celda[0] # accedemos al primer elemento de la lista, que en este caso tiene solo un elemento, el 0
        # print(puntuacion)

        if "/" in puntuacion:
            partes = puntuacion.split("/")
            # print(partes[0])

            if "(" not in partes[0]:
                BC.at[indice, "Ptos_Obs_R1"] = partes[0]
                BC.at[indice, "Tiempo_R1"] = partes[1]
            else:
                ptos_obs = partes[0].split("(")[1][0]
                ptos_tiempo = partes[0].split("(")[1][-6]
                BC.at[indice, "Ptos_Obs_R1"] = ptos_obs
                BC.at[indice, "Ptos_tiempo_R1"] = ptos_tiempo
                BC.at[indice, "Tiempo_R1"] = partes[1]
        else:
            BC.at[indice, "Ptos_Obs_R1"] = puntuacion
            
BC.head(8)

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Ptos_Obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_Obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_Obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,45827,Luna JIMÉNEZ BECHOLD,ESP,165666,SUPREME DU MILON MERCADO DEL AGRICULTOR,SF,ESP,"[79,10]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"79,10",None,None,None,None,None,None,None,None
1,FIN,2,70254,Alejandra MELIAN SANTANA,ESP,167713,IL FOTOGRAFO,PSI,ESP,"[80,58]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"80,58",None,None,None,None,None,None,None,None
2,FIN,3,45827,Luna JIMÉNEZ BECHOLD,ESP,162922,OHIO LV VAN T BIEZENHOF,BWP,ESP,"[92,42]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"92,42",None,None,None,None,None,None,None,None
3,FIN,4,155208,Loic MOLINA MÁRQUEZ,ESP,155995,GWYNETH,KWPN,ESP,"[96,75]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"96,75",None,None,None,None,None,None,None,None
4,FIN,5,273709,Saul Alejandro SÁNCHEZ ROJAS,ESP,168850,GIMORA,KWPN,ESP,"[101,53]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"101,53",None,None,None,None,None,None,None,None
5,FIN,6,38323,Nicole STAIGER,ESP,146915,BENALUP C,CDE,ESP,"[105,58]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"105,58",None,None,None,None,None,None,None,None
6,FIN,7,33381,Isabel PALLERO DÁVILA,ESP,90270,MARISCAL DEL PAS,CDE,ESP,"[115,45]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"115,45",None,None,None,None,None,None,None,None


## Pruebas norma (273.3.3.1): Prueba en dos Mangas

Ambas mangas en Baremo A, ambas con cronómetro.

- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T155509.489.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T163100.768.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T185621.773.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T192327.719.xls

In [236]:
Dos_m = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T155509.489.xls")
Dos_m = modificaciones_generales(Dos_m)
Dos_m
# Dos_m["Puntuacion"] = Dos_m["Puntuacion"].str.split("\n")
# # Dos_m
# Dos_m.iloc[-10,9][0].split("/")[0].split("(")[1].split("-")[1][:-5].strip()

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso
0,FIN,1,5042,Pedro Fernando MATEOS RODRÍGUEZ,ESP,149266,QUIZAS CC,CDE,ESP,"0/67,01\n0/42,65",True,1000,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
1,FIN,2,274244,Alex CODINA LERIA,ESP,155917,COSTELLO DC,CDE,ESP,"0/69,79\n0/46,09",True,800,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
2,FIN,3,132635,Pilar MATEOS DE URBINA,ESP,147592,AZURE DES ROCHES,SF,ESP,"0/71,58\n0/47,71",True,600,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
3,FIN,4,369989,Emmy Pauline KROGER KROGER,ESP,170700,DSP DACENTO H,REITF,ESP,"0/70,10\n0/47,74",True,400,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
4,FIN,5,323398,Alex TEJADA BEROIZ,ESP,116440,APOLO DE EQUIANPE,CDE,ESP,"4(4 Obs - 0 Tpo)/66,87\n0/44,20",True,300,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
5,FIN,6,15975,Alvaro SANCHEZ-ARCILLA ARTIÑANO,ESP,156003,CHAMP BLUE,REITF,ESP,"0/69,20\n4(4 Obs - 0 Tpo)/44,83",True,200,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
6,FIN,7,5018,Javier José MATEOS RODRÍGUEZ,ESP,143177,LEVIS C,HANN,ESP,"4(4 Obs - 0 Tpo)/67,53\n0/47,08",True,150,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
7,FIN,8,14425,Carmen GARCIA-TORRES SOLANO,ESP,159657,FOR JEWEL EDM Z,BWP,ESP,"4(4 Obs - 0 Tpo)/67,18\n0/48,14",True,140,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
8,FIN,9,323398,Alex TEJADA BEROIZ,ESP,126325,TABATA DE OLID,CDE,ESP,"1(0 Obs - 1 Tpo)/73,33\n4(4 Obs - 0 Tpo)/48,75",True,135,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...
9,FIN,10,14425,Carmen GARCIA-TORRES SOLANO,ESP,159656,QUOTE FOR PLEASURE,BWP,ESP,"4(4 Obs - 0 Tpo)/67,38\n4(4 Obs - 0 Tpo)/47,90",True,130,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...


In [235]:

Dos_m

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Concurso:,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,NaN,NaN,NaN,NaN,NaN,NaN
2,Fecha:,2025-03-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,Prueba:,"1,45 (GP) - Dos Mangas (273.3.4)",NaN,NaN,NaN,NaN,NaN,NaN
4,Categoría:,CSI****,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
65,RET,10071629,Nicola POHL (GER),105LA15,ARLO DE BLONDEL (GBR),--,RET,0
66,RET,10137362,Emma CRAWFORD (GBR),107DV35,DUKES OF HAZZARD (GBR),--,RET,0
67,RET,10041949,Andrej PAVLOVIC (SLO),108ME31,KING (NED),--,RET,0
68,RET,10002578,Radovan SILLO (SVK),107KT56,CARLO VAN KLAPSCHEUT Z (BEL),--,RET,0


## Pruebas norma (273.3.4): Prueba en dos Mangas

Ambas mangas en Baremo A, ambas con cronómetro.

In [152]:
Dos_m = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T165920.069.xls")
# Dos_m = modificaciones_generales(Dos_m)
# Dos_m[["Puntuacion_1", "Puntuacion_2"]] = Dos_m["Puntuacion"].str.split("\n", expand = True)
# Dos_m["Puntuacion_3"] = None
# # Dos_m = Dos_m.drop("Puntuacion", axis = 1)
# Dos_m["Puntuacion"] = Dos_m["Puntuacion"].str.split("\n")
Dos_m.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Concurso:,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,NaN,NaN,NaN,NaN,NaN,NaN
2,Fecha:,2025-03-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,Prueba:,"1,45 (GP) - Dos Mangas (273.3.4)",NaN,NaN,NaN,NaN,NaN,NaN
4,Categoría:,CSI****,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Puesto,Jinete/Amazona,NaN,Caballo,NaN,NaN,Puntuación,Premio
8,NaN,FEI,Nombre,FEI,Nombre,Raza,NaN,NaN
9,1,10016473,Penelope LEPREVOST (FRA),106VL82,DJAGGER SEMILLY (FRA),--,"0/59,15",7050


In [149]:
for indice, row in Dos_m.iterrows():
    celda = row["Puntuacion"]
        
    if len(celda) == 1:
        puntuacion = celda[0] # accedemos al primer elemento de la lista, que en este caso tiene solo un elemento, el 0
        # print(puntuacion)

        if "/" in puntuacion:
            partes = puntuacion.split("/")
            # print(partes[0])

            if "(" not in partes[0]:
                Dos_m.at[indice, "Ptos_Obs_R1"] = partes[0]
                Dos_m.at[indice, "Tiempo_R1"] = partes[1]
            elif "(" in partes[0]:
                ptos_obs = partes[0].split("(")[1].split("-")[0][:-5].strip()
                ptos_tiempo = partes[0].split("(")[1].split("-")[1][:-5].strip()
                Dos_m.at[indice, "Ptos_Obs_R1"] = ptos_obs
                Dos_m.at[indice, "Ptos_tiempo_R1"] = ptos_tiempo
                Dos_m.at[indice, "Tiempo_R1"] = partes[1]
        else:
            Dos_m.at[indice, "Ptos_Obs_R1"] = puntuacion

    elif len(celda) == 2:
        puntuacion_r1 = celda[0]
        puntuacion_r2 = celda[1]

        if "/" in puntuacion_r1:
            partes_r1 = puntuacion_r1.split("/")
            if "(" not in partes_r1[0]:
                Dos_m.at[indice, "Ptos_Obs_R1"] = partes_r1[0]
                Dos_m.at[indice, "Tiempo_R1"] = partes_r1[1]

            elif "(" in partes_r1[0]:
                ptos_obs_r1 = partes_r1[0].split("(")[1].split("-")[0][:-5].strip()
                ptos_tiempo_r1 = partes_r1[0].split("(")[1].split("-")[1][:-5].strip()
                Dos_m.at[indice, "Ptos_Obs_R1"] = ptos_obs_r1
                Dos_m.at[indice, "Ptos_tiempo_R1"] = ptos_tiempo_r1
                Dos_m.at[indice, "Tiempo_R1"] = partes_r1[1]
        else:
            BA_sc.at[indice, "Ptos_Obs_R1"] = puntuacion_r1
        
        if "/" in puntuacion_r2:
            partes_r2 = puntuacion_r2.split("/")
            if "(" not in partes_r2[0]:
                Dos_m.at[indice, "Ptos_Obs_R2"] = partes_r2[0]
                Dos_m.at[indice, "Tiempo_R2"] = partes_r2[1]

            elif "(" in partes_r2[0]:
                ptos_obs_r2 = partes_r2[0].split("(")[1].split("-")[0][:-5].strip()
                ptos_tiempo_r2 = partes_r2[0].split("(")[1].split("-")[1][:-5].strip()
                Dos_m.at[indice, "Ptos_Obs_R2"] = ptos_obs_r2
                Dos_m.at[indice, "Ptos_tiempo_R2"] = ptos_tiempo_r2
                Dos_m.at[indice, "Tiempo_R2"] = partes_r2[1]
        else:
            Dos_m.at[indice, "Ptos_Obs_R2"] = puntuacion_r2


Dos_m.head(15)

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Ptos_Obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_Obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_Obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,10016473,Penelope LEPREVOST,FRA,106VL82,DJAGGER SEMILLY,None,FRA,"[0/59,15]",True,7050,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"59,15",None,None,None,None,None,None
1,FIN,2,10150533,Finn BOEREKAMP,NED,106RM98,JAY JAY ES,KWPN,ESP,"[0/59,68]",True,5640,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"59,68",None,None,None,None,None,None
2,FIN,3,10045115,Anthony BOURQUARD,SUI,108CC64,FLANAGAN SEMILLY,None,SUI,"[0/60,28]",True,4230,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"60,28",None,None,None,None,None,None
3,FIN,4,10058729,Rodrigo Giesteira ALMEIDA,POR,107DF31,COMME LE COEUR,None,GER,"[0/60,49]",True,2820,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"60,49",None,None,None,None,None,None
4,FIN,5,10097112,Sanne THIJSSEN,NED,107FW98,LOLANDRIA Z,None,NED,"[0/61,21]",True,1974,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"61,21",None,None,None,None,None,None
5,FIN,6,10002057,Luca Maria MONETA,ITA,108CQ45,EL CAPONE RK,None,FRA,"[0/62,35]",True,1551,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"62,35",None,None,None,None,None,None
6,FIN,7,10017385,William WHITAKER,GBR,107HD18,MILLFIELD QUICKSTEP,None,GBR,"[0/64,63]",True,1128,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"64,63",None,None,None,None,None,None
7,FIN,8,10098599,William FLETCHER,GBR,108PH34,CALIXTE HEARTBREAKER Z,None,GBR,"[0/64,75]",True,846,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"64,75",None,None,None,None,None,None
8,FIN,9,10185632,Anastasia NIELSEN,MON,105VW34,TAILORMADE CONTOU,None,DEN,"[0/64,81]",True,846,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"64,81",None,None,None,None,None,None
9,FIN,10,10005788,Camila MAZZA SMIT,BRA,107HA61,BARBAROSSA DREE BOEKEN,None,BEL,"[0/66,38]",True,705,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...,0,None,"66,38",None,None,None,None,None,None


# Valores jsons

In [3]:
import json

# Cargar el JSON
with open('../data/data_salto/concursos/concursos_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_concursos = json.load(file)

# Obtener la longitud de referencia
longitud_maxima = max(len(v) for v in datos_concursos.values() if isinstance(v, list))

# Listas a revisar
listas_objetivo = ["Inicio", "Final", "Ambito"]

# Detectar y mostrar valores faltantes
for key in listas_objetivo:
    lista = datos_concursos[key]
    indices_presentes = set(range(len(lista)))
    indices_totales = set(range(longitud_maxima))
    indices_faltantes = sorted(indices_totales - indices_presentes)

    print(f"\n🔎 Valores faltantes en la lista '{key}':")

    for idx in indices_faltantes:
        nombre = datos_concursos["Nombre"][idx] if idx < len(datos_concursos["Nombre"]) else "(sin nombre)"
        fecha_inicio = datos_concursos["Inicio"][idx] if idx < len(datos_concursos["Inicio"]) else "(sin fecha)"
        fecha_final = datos_concursos["Final"][idx] if idx < len(datos_concursos["Final"]) else "(sin fecha)"
        print(f"- Índice {idx} → concurso: {nombre} | Inicio: {fecha_inicio} | Final: {fecha_final}")




🔎 Valores faltantes en la lista 'Inicio':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)

🔎 Valores faltantes en la lista 'Final':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)

🔎 Valores faltantes en la lista 'Ambito':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)


In [4]:
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 1144
La longitud de Fecha es: 1144
La longitud de Prueba es: 1144
La longitud de Categoría es: 1142
La longitud de Número es: 1141
La longitud de Concurso es: 1144


In [7]:
import json

# Cargar datos de pruebas
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Longitud total según el campo más largo
longitud_total = len(datos_pruebas["Concurso"])

# Campos a verificar
campos_objetivo = ["Prueba", "Categoría", "Fecha"]  # puedes añadir más si quieres

# Verificar faltantes y mostrar con fecha si existe
for campo in campos_objetivo:
    if campo not in datos_pruebas:
        continue
    
    lista = datos_pruebas[campo]
    indices_faltantes = set(range(longitud_total)) - set(range(len(lista)))
    
    if indices_faltantes:
        print(f"\n🔍 Faltantes en '{campo}':")
        for i in sorted(indices_faltantes):
            fecha = datos_pruebas["Fecha"][i] if i < len(datos_pruebas["Fecha"]) else "(sin fecha)"
            concurso = datos_pruebas["Concurso"][i] if i < len(datos_pruebas["Concurso"]) else "(sin concurso)"
            print(f"- Índice {i} → concurso: '{concurso}' | fecha: {fecha}")





🔍 Faltantes en 'Prueba':
- Índice 14957 → concurso: 'CSI2* LIER (BEL)' | fecha: 115-125 - Two phases, both atc T A
- Índice 14958 → concurso: 'CSI2* LIER (BEL)' | fecha: CSI YH*
- Índice 14959 → concurso: 'CSI2* LIER (BEL)' | fecha: Disponibles
- Índice 14960 → concurso: 'CSI2* LIER (BEL)' | fecha: Salto de Obstáculos
- Índice 14961 → concurso: 'CSI2* LIER (BEL)' | fecha: 16/12/2020
- Índice 14962 → concurso: 'CSI2* LIER (BEL)' | fecha: 16/12/2020
- Índice 14963 → concurso: 'CSI2* LIER (BEL)' | fecha: 17/12/2020
- Índice 14964 → concurso: 'CSI2* LIER (BEL)' | fecha: 17/12/2020
- Índice 14965 → concurso: 'CSIYH LIER (BEL)' | fecha: 18/12/2020
- Índice 14966 → concurso: 'CSIYH LIER (BEL)' | fecha: 18/12/2020
- Índice 14967 → concurso: 'CSIYH LIER (BEL)' | fecha: 19/12/2020
- Índice 14968 → concurso: 'CSIYH LIER (BEL)' | fecha: 19/12/2020
- Índice 14969 → concurso: 'CSIYH LIER (BEL)' | fecha: 20/12/2020
- Índice 14970 → concurso: 'CSI2* LIER (BEL)' | fecha: (sin fecha)
- Índice 14971 → c

In [9]:
with open('../data/data_salto/concursos/concursos_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 149
La longitud de Categoría es: 149
La longitud de Provincia es: 149
La longitud de Localidad es: 149
La longitud de Disciplina es: 149
La longitud de Federación es: 149
La longitud de Resultados es: 149
La longitud de País es: 149
La longitud de Inicio es: 148
La longitud de Final es: 148
La longitud de Ambito es: 148


In [10]:
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 1144
La longitud de Fecha es: 1144
La longitud de Prueba es: 1144
La longitud de Categoría es: 1142
La longitud de Número es: 1141
La longitud de Concurso es: 1144


# Completo

In [38]:
with open('../data/data_completo/pruebas/pruebas_completo_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 561
La longitud de Fecha es: 561
La longitud de Prueba es: 561
La longitud de Categoría es: 561
La longitud de Número es: 561
La longitud de Concurso es: 561


In [39]:
with open('../data/data_completo/pruebas/pruebas_completo_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 496
La longitud de Fecha es: 496
La longitud de Prueba es: 496
La longitud de Categoría es: 496
La longitud de Número es: 496
La longitud de Concurso es: 496


In [40]:
with open('../data/data_completo/concursos/concursos_completo_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 480
La longitud de Categoría es: 480
La longitud de Provincia es: 480
La longitud de Localidad es: 480
La longitud de Disciplina es: 480
La longitud de Federación es: 480
La longitud de Resultados es: 480
La longitud de País es: 480
La longitud de Inicio es: 477
La longitud de Final es: 477
La longitud de Ambito es: 477


In [41]:
with open('../data/data_completo/concursos/concursos_completo_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 223
La longitud de Categoría es: 223
La longitud de Provincia es: 223
La longitud de Localidad es: 223
La longitud de Disciplina es: 223
La longitud de Federación es: 223
La longitud de Resultados es: 223
La longitud de País es: 223
La longitud de Inicio es: 223
La longitud de Final es: 223
La longitud de Ambito es: 223
